In [ ]:
import numpy as np
import cv2
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 

In [ ]:
train = np.zeros([1660,256,256,3], dtype = np.float32)
labels_train = np.zeros(1660)

In [ ]:
def load_video(path, max_frames=30, resize=(256, 256)):
    cap = cv2.VideoCapture(path)
    i = 0
    frames = np.zeros([256,256,3], dtype = np.float32)
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frame = frame/255
            i += 1
            if i == frame_1:
                frames = frame
                break
    finally:
        cap.release()
    return np.array(frames)

def cut_save(main_dir,mod):
    i = 0
    for x in os.listdir(main_dir):
        print(x)
        td = main_dir+x+'/'
        for file in os.listdir(td):
            fl = os.path.join(td, file)
            print(fl)
            videos = load_video(fl)
            if mod == 'train':
                train[i] = (videos)
                
                if x =='real':
                    labels_train[i] = 1
                elif x == 'fake':
                    labels_train[i] = 0
                i +=1

In [ ]:
DATA_DIR = '../Focus/Data/'
cut_save(DATA_DIR,"train")

In [ ]:
print(labels_train[500])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train,labels_train,test_size=0.02, random_state=42)

In [ ]:
test_len = y_test.shape
print(test_len)

In [ ]:
plt.imshow(X_train[113])
plt.show()
print(y_train[112])

In [1]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import callbacks

In [2]:
import tensorflow as tf
layers = tf.keras.layers
models = tf.keras.models
losses = tf.keras.losses
optimizers = tf.keras.optimizers 
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks
layers = tf.keras.layers
models = tf.keras.models
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
losses = tf.keras.losses
optimizers = tf.keras.optimizers 
metrics = tf.keras.metrics
utils = tf.keras.utils
callbacks = tf.keras.callbacks
plot_model = tf.keras.utils.plot_model

In [3]:
class Classifier:
    def __init__():
        self.model = 0
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)

In [16]:
# Create a MesoNet class using the Classifier

class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(learning_rate = learning_rate)
        self.model.compile(optimizer = optimizer,
                           loss = 'mae',
                           metrics = ['accuracy'])
    
    def init_model(self): 
        x = Input(shape = (256,256,3))
        
        x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return Model(inputs = x, outputs = y)

In [17]:
class AccuracyHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.val_acc = []
        self.loss = []
        self.val_loss = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

history = AccuracyHistory()
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=8,min_delta=1e-5, verbose=0, mode='min')
mcp_save = callbacks.ModelCheckpoint('../Focus/weights/Video1.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='val_loss',patience=1, verbose=2,factor=0.5,min_lr=0.0000001)

In [18]:
num_classes = 2
meso = Meso4()
meso.load('../Focus/weights/Meso4_DF.h5')
cnn = models.Sequential()
cnn.add(meso.model)
cnn.add(layers.Flatten())
#cnn.add(layers.Dense(1024, activation='relu'))
#cnn.add(layers.Dropout(0.3))
#cnn.add(layers.Dense(512, activation='relu'))
#cnn.add(layers.Dropout(0.3))
#cnn.add(layers.LSTM(40))
# define LSTM model
model = models.Sequential()
model.add(layers.TimeDistributed(cnn,  input_shape=(30, 256, 256, 3)))
model.add(layers.LSTM(40 , return_sequences=True))
#model.add(layers.Dense(num_classes, activation="sigmoid"))
#model.add(layers.Dropout(0.3))
model.add(layers.TimeDistributed(layers.Dense(160, activation='relu')))
model.add(layers.GlobalAveragePooling1D(name="globale"))
'''
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
'''
model.add(layers.Dense(num_classes, activation="sigmoid" , name="last"))
adam = optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model.load_weights('Deepfake.hdf5')
rms = optimizers.RMSprop()
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_7 (TimeDist (None, 30, 1)             28073     
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 40)            6720      
_________________________________________________________________
time_distributed_8 (TimeDist (None, 30, 160)           6560      
_________________________________________________________________
globale (GlobalAveragePoolin (None, 160)               0         
_________________________________________________________________
last (Dense)                 (None, 2)                 322       
Total params: 41,675
Trainable params: 41,579
Non-trainable params: 96
_________________________________________________________________


In [ ]:


# meso.model.summary()

In [ ]:
epochs = 100
batch_size = 4

In [ ]:
meso.model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.1,
          callbacks=[mcp_save,history])

In [ ]:
X = X_test
y = y_test
pred = meso.predict(X)
count1 = 0
for i in range(test_len[0]):
    print(f"\nCorrect prediction: {round(pred[i][0])==y[i]}")
    if(round(pred[i][0])==y[i]):
        count1 += 1;
print(count1/test_len[0])

In [ ]:
test = np.zeros([518,256,256,3], dtype = np.float32)

In [ ]:
df = pd.read_csv("../Focus/sample_submission.csv")

In [ ]:
print(df)

In [ ]:
def prepare_all_videos(df, root_dir, frame):
    num_samples = len(df)
    video_paths = df["id"].values.tolist()
    # For each video.
    for idx, path in enumerate(video_paths):
#         print(idx,path)
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path),frame)
        test[idx] = frames
        


In [ ]:
frm = 1
num_f = 0 
sum1 = np.zeros([518])
label = np.zeros([518])
while frm <= 25:
    print(frm)
    prepare_all_videos(df, "../Focus/test_face/" ,frm)
    num_f += 1
    frm += 5
    
    pred = meso.predict(test)
    for i in range(518):
        sum1[i] += pred[i][0]
for i in range(518):
    label[i] = sum1[i]/num_f


In [ ]:
# del pred

In [ ]:
# pred = meso.predict(test)

In [ ]:
# print(pred.shape)
# pred_class_1 = np.zeros([518])
# for i in range(518):
#     pred_class_1[i] = pred[i][0]
# print(pred_class_1)

In [ ]:
# label = pred_class_1 

In [ ]:
for i in range(518):
    label[i] = round(label[i])
label = label.astype(int)
print(label)

In [ ]:
# print(pred.shape)
# pred_class = np.zeros([518])
# for i in range(518):
#     pred_class[i] = round(pred[i][0])
# label = pred_class.astype(int)
# print(label)

In [ ]:
ids = df["id"].values.tolist()
list_of_tuples = list(zip(ids, label)) 
df2 = pd.DataFrame(list_of_tuples, columns = ['id', 'label']) 
df2.to_csv('../Focus/submission_13.csv',index=False)